In [ ]:
import sys
sys.path.insert(0, "./src")

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from imageio import imread
import sagemaker
from sagemaker import get_execution_role
import project_utils
from imageio import imread
from PIL import Image
import glob
from sagemaker.tensorflow import TensorFlow, TensorFlowModel
import tensorflow as tf
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix

%load_ext autoreload

%autoreload 2
%aimport project_utils
%matplotlib inline


# Sagemaker preparation

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


# general prefix
prefix='kaggle-plant-prediction'

# *unique* train/test prefixes
train_prefix   = '{}/{}'.format(prefix, 'train_data')
validation_prefix    = '{}/{}'.format(prefix, 'val')

In [ ]:
train_prefix

In [ ]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [ ]:
display(df_train.head())

display(df_test.head())

# Change file structure and upload data to S3

In [ ]:
project_utils.change_file_structure(df_train)

In [ ]:
bucket_path  = sagemaker_session.upload_data('./train_data', bucket=bucket, key_prefix=prefix)

In [ ]:
train_input_path = "s3://{}/{}/train/".format(bucket, prefix)
validation_input_path = "s3://{}/{}/val/".format(bucket, prefix)


# Sagemaker training jobs

In [ ]:

s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

tf_estimator = TensorFlow(entry_point='./src/model_new.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.12', 
                          py_version='py3'
                         )


In [ ]:
tf_estimator.fit({'train': train_input_path, 'val': validation_input_path})

# Sagemaker evaluation

In [ ]:
from sagemaker.tensorflow import TensorFlowPredictor

# Please note: endpoint is deployed through the GUI, you need to choose your own endpoint's name
predictor = TensorFlowPredictor('kaggle-plantnet-detection')

In [ ]:
val_imgs = glob.glob("./train_data/val/*/*.jpg")

In [ ]:
val_memmap = np.memmap("val.dat", dtype='float32', mode='w+', shape=(len(val_imgs),224,224,3))

for i in range(len(val_imgs)):
    img = imread(val_imgs[i])
    img = np.asarray(Image.fromarray(img).resize((224,224))).reshape(224,224,3) /255.
    val_memmap[i] = img

In [ ]:
y_tmp = [img_path.split("/")[-2] for img_path in val_imgs]

y = label_binarize(y_tmp, classes=["healthy", "multiple_diseases", "rust", "scab"])


val_memmap = np.memmap("val.dat", dtype='float32', mode='r', shape=(len(val_imgs),224,224,3))

pred_list = []
for idx in range(0, len(val_memmap), 1):
    prediction = predictor.predict(val_memmap[idx:idx+1])
    pred_list.append(prediction)


In [ ]:
y_score = np.asarray([pred["predictions"][0] for pred in pred_list])

In [ ]:
# based on: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

n_classes = y.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure()
plt.plot(fpr[2], tpr[2], color='darkblue',
         lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='darkgray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
deviation = y_score.argmax(axis=1) - y.argmax(axis=1)

In [ ]:
target_classes = ["healthy", "multiple_diseases", "rust", "scab"]

In [ ]:
img_paths = [val_imgs[i] for i in np.argwhere(deviation != 0).flatten()]
img_y_true = [y[i].argmax() for i in np.argwhere(deviation != 0).flatten()]
img_y_true = [target_classes[i] for i in img_y_true]
img_y_pred = [y_score[i].argmax() for i in np.argwhere(deviation != 0).flatten()]
img_y_pred = [target_classes[i] for i in img_y_pred]



In [ ]:
idx = 4

plt.imshow(imread(img_paths[idx]))
print("Actual: {}".format(img_y_true[idx]))
print("Predicted: {}".format(img_y_pred[idx]))

In [ ]:
idx = 7

plt.imshow(imread(img_paths[idx]))
print("Actual: {}".format(img_y_true[idx]))
print("Predicted: {}".format(img_y_pred[idx]))

In [ ]:
confusion_matrix(y.argmax(axis=1), y_score.argmax(axis=1))

# Kaggle predictions

In [ ]:

test_imgs = glob.glob("./images/Test*.jpg")

pred_memmap = np.memmap("pred.dat", dtype='float32', mode='w+', shape=(len(test_imgs),224,224,3))

for i in range(len(test_imgs)):
    img = imread(test_imgs[i])
    img = np.asarray(Image.fromarray(img).resize((224,224))).reshape(224,224,3) /255.
    pred_memmap[i] = img

In [ ]:
pred_memmap = np.memmap("pred.dat", dtype='float32', mode='r', shape=(len(test_imgs),224,224,3))


pred_list = []
for idx in range(0, len(pred_memmap), 1):
    prediction = predictor.predict(pred_memmap[idx:idx+1])
    pred_list.append(prediction)

In [ ]:
idx = 0
plt.imshow(np.asarray(pred_memmap[idx]))
display(pred_list[idx])

In [ ]:
name = pd.DataFrame(np.asarray([name.split("/")[-1].split(".")[0] for name in test_imgs]), columns=["image_id"])
pred = pd.DataFrame(np.asarray([pred["predictions"][0] for pred in pred_list]), columns=["healthy", "multiple_diseases", "rust", "scab"])

In [ ]:
name.join(pred).to_csv("submission.csv", index=None)